In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data
customers = pd.read_csv('../data/processed/customer_segments.csv')
diamond = pd.read_csv('../data/processed/diamond_customers.csv')
high_risk = pd.read_csv('../data/processed/high_value_churn_risk.csv')

def create_enhanced_executive_dashboard(customers, diamond_customers):
        
    # Calculate key metrics
    total_customers = len(customers)
    total_value = customers['clv_1yr'].sum()
    avg_value = customers['clv_1yr'].mean()
    high_value_customers = len(diamond_customers)
    high_value_percentage = high_value_customers / total_customers * 100
    high_value_contribution = diamond_customers['clv_1yr'].sum() / total_value * 100
    
    # Calculate future projected portfolio value (10% growth for high-value, 2% for others)
    future_high_value = diamond_customers['clv_1yr'].sum() * 1.10
    future_regular_value = (total_value - diamond_customers['clv_1yr'].sum()) * 1.02
    future_total = future_high_value + future_regular_value
    growth_potential = future_total - total_value
    
    # Create executive dashboard
    fig = make_subplots(
        rows=3, cols=2,
        specs=[
            [{"type": "indicator"}, {"type": "indicator"}],
            [{"type": "indicator"}, {"type": "indicator"}],
            [{"type": "pie", "colspan": 2}, None],
        ],
        subplot_titles=(
            "Total Portfolio Value", "Average Customer Lifetime Value", 
            "High-Value Customer Percentage", "Portfolio Growth Potential",
            "Portfolio Value Distribution"
        ),
        vertical_spacing=0.1
    )
    
    # Add KPI indicators
    fig.add_trace(
        go.Indicator(
            mode="number+delta",
            value=total_value,
            number={"prefix": "$", "font": {"size": 40}},
            delta={"reference": total_value * 0.95, "relative": True},
            title={"text": "Total Portfolio Value"},
            domain={"row": 0, "column": 0}
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Indicator(
            mode="number+delta",
            value=avg_value,
            number={"prefix": "$", "font": {"size": 40}},
            delta={"reference": avg_value * 0.95, "relative": True},
            title={"text": "Average CLV"},
            domain={"row": 0, "column": 1}
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Indicator(
            mode="number+gauge",
            value=high_value_percentage,
            number={"suffix": "%", "font": {"size": 40}},
            gauge={
                "axis": {"range": [0, 30]},
                "bar": {"color": "darkblue"},
                "steps": [
                    {"range": [0, 5], "color": "lightgray"},
                    {"range": [5, 10], "color": "gray"},
                    {"range": [10, 30], "color": "lightblue"}
                ],
                "threshold": {
                    "line": {"color": "red", "width": 4},
                    "thickness": 0.75,
                    "value": 10
                }
            },
            title={"text": "High-Value Customer %"},
            domain={"row": 1, "column": 0}
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Indicator(
            mode="number+delta",
            value=growth_potential,
            number={"prefix": "$", "font": {"size": 40}},
            delta={"reference": 0, "relative": False, "valueformat": "+$,.0f"},
            title={"text": "Growth Potential (Next Year)"},
            domain={"row": 1, "column": 1}
        ),
        row=2, col=2
    )
    
    # Add portfolio value distribution pie chart
    labels = ['High-Value Segment', 'Core Segment', 'Low-Value Segment']
    values = [
        diamond_customers['clv_1yr'].sum(),
        customers[(customers['clv_1yr'] > customers['clv_1yr'].median()) & 
                 (~customers['client_id'].isin(diamond_customers['client_id']))]['clv_1yr'].sum(),
        customers[customers['clv_1yr'] <= customers['clv_1yr'].median()]['clv_1yr'].sum()
    ]
    
    fig.add_trace(
        go.Pie(
            labels=labels,
            values=values,
            hole=0.4,
            marker_colors=['#2E86C1', '#85C1E9', '#D6EAF8'],
            textinfo='label+percent',
            textposition='inside',
            insidetextorientation='radial',
            pull=[0.1, 0, 0],
            domain={"row": 2, "column": 0}
        ),
        row=3, col=1
    )
    
    # Update layout with annotations for business context
    fig.update_layout(
        height=800,
        width=1000,
        title_text="Executive Dashboard: Customer Portfolio Value",
        annotations=[
            dict(
                x=0.5,
                y=-0.1,
                showarrow=False,
                text="High-value customers represent only " + 
                     f"{high_value_percentage:.1f}% of customers but contribute " +
                     f"{high_value_contribution:.1f}% of portfolio value",
                xref="paper",
                yref="paper",
                font=dict(size=14)
            )
        ]
    )
    
    return fig


In [2]:
# Usage
exec_dashboard = create_enhanced_executive_dashboard(customers, diamond)
exec_dashboard.show()

KeyError: 'customer_id'